<a href="https://colab.research.google.com/github/RayNCode/code_collab/blob/main/Main_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here I try to compile everything I have done so far. I merged all the datasets together and I'm trying to keep only the relevant variables for our model.

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np

In [23]:
learn_dataset_job = pd.read_csv("/content/learn_dataset_job.csv")
work_desc = pd.read_csv("/content/code_work_description_map.csv")
merged_df = pd.merge(learn_dataset_job, work_desc, left_on='work_description', right_on='N3', how='left')

# Convertir les colonnes N2, N1 et N3 en chaînes de caractères
merged_df['N2'] = merged_df['N2'].astype(str)
merged_df['N1'] = merged_df['N1'].astype(str)
merged_df['N3'] = merged_df['N3'].astype(str)

# Remplir les valeurs manquantes dans N2 en fonction de N3
merged_df['N2'].fillna(merged_df['N3'].str[:-2], inplace=True)
# Remplir les valeurs manquantes dans N1 en fonction de N2
merged_df['N1'].fillna(merged_df['N2'].str[:-1], inplace=True)

learn_dataset = pd.read_csv("/content/learn_dataset.csv")
data_2 = pd.merge(learn_dataset, merged_df, on="Id", how="left")
print(data_2.shape)
dept_code = pd.read_csv("/content/city_adm.csv")
df_1 = data_2.merge(dept_code, on='insee_code')
print(df_1.shape)
emp_contract = pd.read_csv("/content/learn_dataset_Emp_contract.csv")
df_2 = df_1.merge(emp_contract, on='Id', how='left')
print(df_2.shape)
learn_dataset_sport = pd.read_csv("/content/learn_dataset_sport.csv")
df_3 = df_2.merge(learn_dataset_sport, on='Id', how='left')
print(df_3.shape)
code_Club = pd.read_csv("/content/code_Club.csv")
df_4 = df_3.merge(code_Club, left_on='Club', right_on='Code', how='left')
print(df_4.shape)
dep = pd.read_csv("/content/departments.csv")
df = df_4.merge(dep, on='dep', how='left')
df.shape

(49993, 23)
(49993, 26)
(49993, 27)
(49993, 28)
(49993, 32)


(49993, 34)

In [24]:
df['Categorie'] = df['Categorie'].astype('object')
df['REG'] = df['REG'].astype('object')
print(df.dtypes)

Id                      int64
insee_code             object
is_student               bool
OCCUPATION_42          object
ACTIVITY_TYPE          object
AGE_2019                int64
household              object
Highest_degree         object
sex                    object
target                 object
employer_category      object
job_category           object
EMPLOYEE_COUNT         object
Terms_of_emp           object
Eco_sect               object
work_description       object
Job_dep                object
Working_hours         float64
WORK_CONDITION         object
EMOLUMENT             float64
N3                     object
N2                     object
N1                     object
Nom de la commune      object
town_type              object
dep                    object
Emp_contract           object
Club                   object
Code                   object
Categorie              object
Nom fédération         object
Nom catégorie          object
Nom du département     object
REG       

In [25]:
df[["EMOLUMENT"]]

,EMOLUMENT
0,12928.0
1,15428.0
2,NaN
3,NaN
4,NaN
...,...
49988,30713.0
49989,NaN
49990,NaN
49991,27321.0


In [31]:
# TYPE2|2	Elèves, étudiants, stagiaires non rémunéré de 14 ans ou plus --> 0 tune
# TYPE2|3	Moins de 14 ans --> 0 personne
# TYPE2|4	Femmes ou hommes au foyer --> 0 tune
# TYPE2|5 Autres inactifs --> 0 tune également
# TYPE2|1	--> retraités/ toutes ces catégories là il faut set 0 à EMOLUMENT avant de faire le modèle de prédiction.
# Il faut également tester si après avoir fait ça il reste des personnes dans la base de données qui n'ont pas de salaire. Ce qui est étrange c'est que les chomeurs n'ont pas d'argent.
# Pourtant ils percoivent de l'argent... A voir si on a envie d'aller + loin là-dessus.

# en gros :
# if 'ACTIVITY_TYPE' != "TYPE1|2":
#                "EMOLUMENT", "N1", "N2" ==0



# Je dois trouver un moyen de parametrer les emoluements et autres variables dépendantes du travail à 0 et ajouter une autre catégorie "None".
# Peut-être le faire après la prédiction justement comme ça on garde le reste.

In [26]:
# Créez une condition pour filtrer les lignes où 'ACTIVITY_TYPE' n'est pas égal à "TYPE1|2"
condition = (df['ACTIVITY_TYPE'] != "TYPE1|1")

# Imputez les valeurs de 'EMOLUMENT' et 'Working_hours' à zéro pour les lignes qui satisfont la condition
df.loc[condition, ['EMOLUMENT', 'Working_hours']] = 0.0

In [27]:
df[["EMOLUMENT", "Working_hours"]]

,EMOLUMENT,Working_hours
0,12928.0,1818.0
1,15428.0,1408.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
49988,30713.0,1823.0
49989,0.0,0.0
49990,0.0,0.0
49991,27321.0,1778.0


In [28]:
categorical_columns = df.select_dtypes(include=['object']).columns
# Remplacer les valeurs manquantes par "None" dans les colonnes catégorielles
df[categorical_columns] = df[categorical_columns].fillna("None")

In [29]:
y = df['target'].copy()
y = np.where(y == 'B', 1, 0)
X = df.drop(["Id", "target", 'insee_code'], axis="columns").copy()

In [30]:
df_students = df[df['ACTIVITY_TYPE'] == "TYPE1|1"]
null_counts = df_students.isnull().sum()
null_counts['Working_hours']

4942

In [31]:
#  useful bc there are some missing values for Working_hours only 19270/24212 observations and EMOLUMENT 19279/24212.
# Therefore, we have to estimate them by using a Random Forest Regressor.

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# Créez l'IterativeImputer en spécifiant le modèle de régression (RandomForestRegressor)
imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=20, random_state=0)

# Sélectionnez les colonnes numériques à imputer
columns_to_impute = ['Working_hours', 'EMOLUMENT']

# Appliquez l'imputation uniquement sur les colonnes sélectionnées
X[columns_to_impute] = imputer.fit_transform(X[columns_to_impute])

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [32]:
# Vérifier si vous avez des valeurs nulles dans tout le DataFrame
X.isna().sum()

is_student            0
OCCUPATION_42         0
ACTIVITY_TYPE         0
AGE_2019              0
household             0
Highest_degree        0
sex                   0
employer_category     0
job_category          0
EMPLOYEE_COUNT        0
Terms_of_emp          0
Eco_sect              0
work_description      0
Job_dep               0
Working_hours         0
WORK_CONDITION        0
EMOLUMENT             0
N3                    0
N2                    0
N1                    0
Nom de la commune     0
town_type             0
dep                   0
Emp_contract          0
Club                  0
Code                  0
Categorie             0
Nom fédération        0
Nom catégorie         0
Nom du département    0
REG                   0
dtype: int64

In [33]:
X.columns

Index(['is_student', 'OCCUPATION_42', 'ACTIVITY_TYPE', 'AGE_2019', 'household',
       'Highest_degree', 'sex', 'employer_category', 'job_category',
       'EMPLOYEE_COUNT', 'Terms_of_emp', 'Eco_sect', 'work_description',
       'Job_dep', 'Working_hours', 'WORK_CONDITION', 'EMOLUMENT', 'N3', 'N2',
       'N1', 'Nom de la commune', 'town_type', 'dep', 'Emp_contract', 'Club',
       'Code', 'Categorie', 'Nom fédération', 'Nom catégorie',
       'Nom du département', 'REG'],
      dtype='object')

In [34]:
X

,is_student,OCCUPATION_42,ACTIVITY_TYPE,AGE_2019,household,Highest_degree,sex,employer_category,job_category,EMPLOYEE_COUNT,...,town_type,dep,Emp_contract,Club,Code,Categorie,Nom fédération,Nom catégorie,Nom du département,REG
0,True,csp_6_8,TYPE1|1,20,TYPMR1-1,edu.1.4,Male,ct_9,A,tr_3,...,Commune simple,01,COND.1.1,None,None,None,None,None,Ain,84
1,False,csp_6_5,TYPE1|1,48,TYPMR4-1,edu.1.3,Male,ct_9,O,tr_5,...,Chef-lieu canton,01,COND.1.6,fede.109,fede.109,1.0,Fédération Française d'équitation,Fédération unisport olympique,Ain,84
2,False,csp_8_1,TYPE1|2,37,TYPMR4-4,edu.1.8,Female,None,None,None,...,Chef-lieu canton,01,None,fede.102,fede.102,1.0,Fédération Française des sociétés d'aviron,Fédération unisport olympique,Ain,84
3,False,csp_4_7,TYPE1|2,65,TYPMR3-1,edu.1.6,Male,None,None,None,...,Chef-lieu canton,01,None,None,None,None,None,None,Ain,84
4,True,csp_8_4,TYPE2|2,22,TYPMR1-2,edu.1.4,Female,None,None,None,...,Chef-lieu canton,01,None,None,None,None,None,None,Ain,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49988,False,csp_4_6,TYPE1|1,57,TYPMR1-2,edu.1.3,Female,ct_4,O,tr_5,...,Chef-lieu canton,95,COND.1.6,fede.103,fede.103,1.0,Fédération Française de badminton,Fédération unisport olympique,Val-d'Oise,11
49989,False,csp_8_5,TYPE2|5,25,TYPMR4-1,edu.1.4,Male,None,None,None,...,Chef-lieu canton,95,None,fede.106,fede.106,1.0,Fédération Française de boxe,Fédération unisport olympique,Val-d'Oise,11
49990,True,csp_8_4,TYPE2|2,19,TYPMR4-2,edu.1.4,Female,None,None,None,...,Chef-lieu canton,95,None,fede.115,fede.115,1.0,Fédération Française de handball,Fédération unisport olympique,Val-d'Oise,11
49991,False,csp_4_7,TYPE1|1,48,TYPMR4-1,edu.1.7,Male,ct_9,O,tr_1,...,Chef-lieu canton,95,COND.1.6,fede.105,fede.105,1.0,Fédération Française de basketball,Fédération unisport olympique,Val-d'Oise,11


In [ ]:
# J'ai l'impression que certaines catégories n'étaient pas bien comprises. Il aurait fallu que j'écrive "None", pour la colonne Catégorie.

In [ ]:
# Faire le nécessaire pour round up si cela ne marche pas.

In [ ]:
#Voir ici les colonnes que je garde, ou non.

In [ ]:
from sklearn.model_selection import train_test_split
import sklearn
sklearn.set_config(display="diagram")
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)